In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r'../data/Lista_transakcji_nr_0165421859_200723.csv', sep=';', decimal=',', encoding='windows-1250')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 686 entries, 0 to 685
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Data transakcji                     686 non-null    object 
 1   Data księgowania                    667 non-null    object 
 2   Dane kontrahenta                    686 non-null    object 
 3   Tytuł                               686 non-null    object 
 4   Nr rachunku                         612 non-null    object 
 5   Nazwa banku                         96 non-null     object 
 6   Szczegóły                           667 non-null    object 
 7   Nr transakcji                       667 non-null    object 
 8   Kwota transakcji (waluta rachunku)  667 non-null    float64
 9   Waluta                              667 non-null    object 
 10  Kwota blokady/zwolnienie blokady    19 non-null     float64
 11  Waluta.1                            19 non-nu

In [4]:
def clean_data(df):
    # Drop columns: 'Unnamed: 20', 'Waluta' and 10 other columns
    df = df.drop(columns=['Unnamed: 20', 'Waluta', 'Kwota blokady/zwolnienie blokady', 'Waluta.1', 'Kwota płatności w walucie', 'Waluta.2', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'])
    # Drop column: 'Nazwa banku'
    df = df.drop(columns=['Nazwa banku'])
    # Drop column: 'Data księgowania'
    df = df.drop(columns=['Data księgowania'])
    # Drop rows with missing data in column: 'Kwota transakcji (waluta rachunku)'
    df = df.dropna(subset=['Kwota transakcji (waluta rachunku)'])
    # Change column type to datetime64[ns] for column: 'Data transakcji'
    df = df.astype({'Data transakcji': 'datetime64[ns]'})
    # Drop column: 'Szczegóły'
    df = df.drop(columns=['Szczegóły'])
    # Change column type to float16 for column: 'Kwota transakcji (waluta rachunku)'
    df = df.astype({'Kwota transakcji (waluta rachunku)': 'float'})
    # Rename column 'Kwota transakcji (waluta rachunku)' to 'amount'
    df = df.rename(columns={'Kwota transakcji (waluta rachunku)': 'amount'})
    # Rename column 'Nr transakcji' to 'transaction_no'
    df = df.rename(columns={'Nr transakcji': 'transaction_no'})
    # Rename column 'Nr rachunku' to 'account_number'
    df = df.rename(columns={'Nr rachunku': 'account_number'})
    # Rename column 'Tytuł' to 'title'
    df = df.rename(columns={'Tytuł': 'title'})
    # Rename column 'Dane kontrahenta' to 'details'
    df = df.rename(columns={'Dane kontrahenta': 'details'})
    # Rename column 'Data transakcji' to 'date'
    df = df.rename(columns={'Data transakcji': 'date'})
    # Rename column 'account_number' to 'account'
    df = df.rename(columns={'account_number': 'account'})
    # Rename column 'transaction_no' to 'transaction'
    df = df.rename(columns={'transaction_no': 'transaction'})
    # Replace all instances of "'" with "" in column: 'account'
    df['account'] = df['account'].str.replace("'", "", case=False, regex=False)
    # Replace all instances of "'" with "" in column: 'transaction'
    df['transaction'] = df['transaction'].str.replace("'", "", case=False, regex=False)
    # Remove leading and trailing whitespace in column: 'account'
    df['account'] = df['account'].str.strip()
    # Convert text to lowercase in column: 'title'
    df['title'] = df['title'].str.lower()
    # Remove leading and trailing whitespace in column: 'title'
    df['title'] = df['title'].str.strip()
    # Convert text to lowercase in column: 'details'
    df['details'] = df['details'].str.lower()
    # Remove leading and trailing whitespace in column: 'details'
    df['details'] = df['details'].str.strip()
    return df

df_clean = clean_data(df.copy())


In [5]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 667 entries, 0 to 685
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         667 non-null    datetime64[ns]
 1   details      667 non-null    object        
 2   title        667 non-null    object        
 3   account      593 non-null    object        
 4   transaction  667 non-null    object        
 5   amount       667 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 36.5+ KB


In [6]:
df_clean['details'].value_counts().index.tolist()

['lidl lukasinskiego  nowy sacz  pol',
 'jarosław bugaj',
 'allegro  poznan  pol',
 'lidl lukasinskiego, lidl lukasinskiego, nowy sacz',
 'piekarnia oracz a. oracz  nowy sacz',
 'castorama nowy sacz  nowy sacz  pol',
 'jmp s.a. biedronka 4028  nowy sacz',
 'piekarnia oracz  nowy sacz  pol',
 'ludwa dawid jan, i ludwa anna, edmunda ciećkiewicza 11, 33-300 nowy sącz',
 'apteka sloneczna 01  nowy sacz  pol',
 'paypro s.a., pastelowa 8, poznan',
 'paypal *patronite  35314369001 0000',
 'myjnia 24h  nowy sacz  pol',
 'centrum tranow  tarnow  pol',
 'google payment ie ltd you  dublin',
 'orlen stacja nr 138  nowy sacz  pol',
 'market obi 075  nowy sacz  pol',
 'zus centrum obsługi świadczeń dla r, odzin, ul. młynowa 29, 15-404 białystok',
 'paypal *maxtv llc  4029357733 19958',
 'mhr nowy sacz sklep  nowy sacz  pol',
 'poczta polska 03  nowy sacz  pol',
 'orange  warszawa  pol',
 'sądeckie wodociągi sp. z o. o., ul. wincentego pola 22, 33-300 nowy sącz',
 'ludwa dawid, i ludwa anna, babiego 

In [7]:
df = df_clean.copy()

In [8]:
df.head()

,date,details,title,account,transaction,amount
0,2023-07-20,dawid ludwa,bo tak,54102049550000760201606797,202320197209545076,-50.00
3,2023-07-20,"zus centrum obsługi świadczeń dla r, odzin, ul...",świadczenie zus 010000c230722tsw/1014718256 ro...,82102056040000010289963017,202320197203279992,500.00
4,2023-07-19,"ludwa dawid jan, i ludwa anna, edmunda ciećkie...",przelew własny,64105015621000009117462698,202320097202882525,1000.00
11,2023-07-18,lidl lukasinskiego nowy sacz pol,płatność kartą 18.07.2023 nr karty 4246xx8188,1915031/19730,202320197304224746,-56.20
12,2023-07-18,piekarnia oracz a. oracz nowy sacz,płatność kartą 18.07.2023 nr karty 4246xx8188,1915031/19730,202320197301037340,-40.57


In [9]:
df.loc[df["details"].str.contains('Lidl', case=False), "details"] = 'Lidl'
df.loc[df["details"].str.contains('jarosław bugaj', case=False), "details"] = 'Individual - Jarosław Bugaj'
df.loc[df["details"].str.contains('Allegro', case=False), "details"] = 'Allegro'
df.loc[df["details"].str.contains('Apteka', case=False), "details"] = 'Pharmacy'
df.loc[df["details"].str.contains('Biedronka', case=False), "details"] = 'Biedronka'
df.loc[df["details"].str.contains('PayPal', case=False), "details"] = 'PayPal'
df.loc[df["details"].str.contains('Google', case=False), "details"] = 'Google'
df.loc[df["details"].str.contains('Castorama', case=False), "details"] = 'Castorama'
df.loc[df["details"].str.contains('piekarnia oracz', case=False), "details"] = 'Oracz Bakery'
df.loc[df["details"].str.contains('ING Bank', case=False), "details"] = 'ING Bank'
df.loc[df["details"].str.contains('Zabka', case=False), "details"] = 'Zabka'
df.loc[df["details"].str.contains('ludwa dawid jan', case=False), "details"] = 'Individual - Ludwa Family'
df.loc[df["details"].str.contains('paypro s.a.', case=False), "details"] = 'PayPro SA'
df.loc[df["details"].str.contains('Poczta Polska', case=False), "details"] = 'Polish Post'
df.loc[df["details"].str.contains('myjnia 24h', case=False), "details"] = '24h Car Wash'
df.loc[df["details"].str.contains('Orlen', case=False), "details"] = 'Orlen'
df.loc[df["details"].str.contains('ZUS', case=False), "details"] = 'ZUS'
df.loc[df["details"].str.contains('sądeckie wodociągi', case=False), "details"] = 'Sądeckie Waterworks'
df.loc[df["details"].str.contains('Vinted', case=False), "details"] = 'Vinted'
df.loc[df["details"].str.contains('BP', case=False), "details"] = 'BP'
df.loc[df["details"].str.contains('McDonalds', case=False), "details"] = 'McDonalds'
df.loc[df["details"].str.contains('Orange', case=False), "details"] = 'Orange'
df.loc[df["details"].str.contains('market obi', case=False), "details"] = 'Obi Market'
df.loc[df["details"].str.contains('pgnig obrót detaliczny', case=False), "details"] = 'PGNiG Retail'
df.loc[df["details"].str.contains('payu', case=False), "details"] = 'PayU'
df.loc[df["details"].str.contains('Netflix', case=False), "details"] = 'Netflix'
df.loc[df["details"].str.contains('galeria usmiechu', case=False), "details"] = 'Smile Gallery'
df.loc[df["details"].str.contains('chatgpt subscription', case=False), "details"] = 'ChatGPT Subscription'
df.loc[df["details"].str.contains('Pepco', case=False), "details"] = 'Pepco'
df.loc[df["details"].str.contains('Rossmann', case=False), "details"] = 'Rossmann'
df.loc[df["details"].str.contains('Auchan', case=False), "details"] = 'Auchan'
df.loc[df["details"].str.contains('bnp paribas', case=False), "details"] = 'BNP Paribas'
df.loc[df["details"].str.contains('ul.nawojowska', case=False), "details"] = 'Nawojowska St. Address'
df.loc[df["details"].str.contains('Disney', case=False), "details"] = 'Disney'
df.loc[df["details"].str.contains('RTV Euro AGD', case=False), "details"] = 'RTV Euro AGD'
df.loc[df["details"].str.contains('github', case=False), "details"] = 'GitHub'
df.loc[df["details"].str.contains('pyszne.pl', case=False), "details"] = 'Pyszne.pl'
df.loc[df["details"].str.contains('p.h. aika', case=False), "details"] = 'Aika Stand'
df.loc[df["details"].str.contains('cashbill', case=False), "details"] = 'CashBill'
df.loc[df["details"].str.contains('centrum urody', case=False), "details"] = 'Beauty Center'
df.loc[df["details"].str.contains('al capone', case=False), "details"] = 'Al Capone'
df.loc[df["details"].str.contains('al. capone', case=False), "details"] = 'Al Capone'
df.loc[df["details"].str.contains('ul. grodzka', case=False), "details"] = 'Grodzka St. Address'
df.loc[df["details"].str.contains('Empik', case=False), "details"] = 'Empik'
df.loc[df["details"].str.contains('blue media', case=False), "details"] = 'Blue Media'
df.loc[df["details"].str.contains('H&M', case=False), "details"] = 'H&M'
df.loc[df["details"].str.contains('alfa sp. jawna', case=False), "details"] = 'Alfa Partnership'
df.loc[df["details"].str.contains('Jysk', case=False), "details"] = 'Jysk'
df.loc[df["details"].str.contains('aldi sp. z o.o.', case=False), "details"] = 'Aldi'
df.loc[df["details"].str.contains('Media Markt', case=False), "details"] = 'Media Markt'
df.loc[df["details"].str.contains('bluemedia', case=False), "details"] = 'BlueMedia'
df.loc[df["details"].str.contains('yak z nepalu', case=False), "details"] = 'Yak from Nepal'
df.loc[df["details"].str.contains('piekarnia-ciastka39819', case=False), "details"] = 'Bakery & Cookies 39819'
df.loc[df["details"].str.contains('wizz air', case=False), "details"] = 'Wizz Air'
df.loc[df["details"].str.contains('wojas s.a.', case=False), "details"] = 'Wojas SA'
df.loc[df["details"].str.contains('centrum tranow tarnow pol', case=False), "details"] = 'Centrum'
# diagmed
df.loc[df["details"].str.contains('diagmed', case=False), "details"] = 'Diagmed'
# kaufland
df.loc[df["details"].str.contains('kaufland', case=False), "details"] = 'Kaufland'
# lejawa
df.loc[df["details"].str.contains('lejawa', case=False), "details"] = 'Lejawa'
# mobilevikings 
# www.mobileviking.pl 
df.loc[df["details"].str.contains('mobilevikings|mobileviking', case=False), "details"] = 'Mobile Vikings'
# lewiatan 
df.loc[df["details"].str.contains('lewiatan', case=False), "details"] = 'Lewiatan'
# player.pl
df.loc[df["details"].str.contains('player.pl', case=False), "details"] = 'player.pl'
#medicine
df.loc[df["details"].str.contains('medicine', case=False), "details"] = 'medicine'
# barska clinic
df.loc[df["details"].str.contains('barska clinic', case=False), "details"] = 'Barska Clinic'
# ... and so on for the rest of the names
# mhr nowy sącz
df.loc[df["details"].str.contains('mhr nowy sącz', case=False), "details"] = 'Małopolska Hodowla Roślin'
# mhr nowy sacz sklep
df.loc[df["details"].str.contains('mhr nowy sacz sklep', case=False), "details"] = 'Małopolska Hodowla Roślin'
# reserved
df.loc[df["details"].str.contains('reserved', case=False), "details"] = 'Reserved'
# deichmann
df.loc[df["details"].str.contains('deichmann', case=False), "details"] = 'Deichmann'
# bolt
df.loc[df["details"].str.contains('bolt', case=False), "details"] = 'Bolt'
# leroy merlin
df.loc[df["details"].str.contains('leroy merlin', case=False), "details"] = 'Leroy Merlin'
# stacja paliw
df.loc[df["details"].str.contains('stacja paliw', case=False), "details"] = 'Fuel Station'
# lpp sinsay
df.loc[df["details"].str.contains('lpp sinsay', case=False), "details"] = 'Sinsay'
# ryanair  
df.loc[df["details"].str.contains('ryanair', case=False), "details"] = 'Ryanair'
# inpost 
df.loc[df["details"].str.contains('inpost', case=False), "details"] = 'InPost'

# Note: Be careful with short or common words as they might match unwanted text.



In [10]:
# Define your categories
categories = {
    'Supermarket': ['Lidl', 'Biedronka', 'Zabka', 'Auchan', 'Oracz Bakery', 'Aldi', "Centrum", 'Lewiatan', 'Kaufland'],
    'Financial Services': ['PayPal', 'ING Bank', 'paypro s.a.', 'PayU', 'BNP Paribas', 'CashBill', 'Blue Media'],
    'Healthcare': ['Apteka', 'Pharmacy', 'Diagmed', "Barska Clinic", 'spec. praktyka lek'],
    'Retail': ['Pepco', 'Rossmann', 'H&M', 'Jysk', 'Media Markt', 'RTV Euro AGD', 'Empik'],
    'Food Service': ['McDonalds', 'pyszne.pl', 'Al Capone', 'Yak from Nepal', 'Grodzka St. Address', 'Pyszne.pl', 'Bakery & Cookies 39819'],
    'Alcosol': ['Al Capone'],
    'Utility': ['Orange', 'PGNiG Retail', 'Sądeckie Waterworks'],
    'Transport': ['BP', 'Orlen', 'Wizz Air', 'Bolt', 'Fuel Station', '24h Car Wash'],
    'Transport vacation': ['Wizz Air', 'Ryanair'],
    'Ps': ['jarosław bugaj'],
    'Personal': ['Individual', 'Beauty Center', 'Aika Stand', 'Individual - Dawid Ludwa', 'dawid ludwa', 'ludwa dawid, i ludwa anna', 'jolanta wojciechowska' ],
    'Lejawa': ['lejawa'],
    'Clothes': ['H&M', 'Wojas SA', 'medicine', 'Reserved', 'Vinted', 'Deichmann', 'Sinsay'],
    'Home Improvement': ['Castorama', 'Obi Market', 'Małopolska Hodowla Roślin', 'Leroy Merlin'],
    'Entertainment': ['Netflix', 'Disney', 'player.pl'],
    'Technology': ['Google', 'GitHub', 'Mobile Vikings', 'ChatGPT Subscription'],
    'Postal Services': ['Polish Post', 'InPost'],
    'Other': [],
    'Bills': ['ZUS'],
    'Net': ['Allegro', 'PayPro SA', 'BlueMedia'],
    'dentist': ['Smile Gallery'],

}

# Assign categories to each contractor
for category, keywords in categories.items():
    for keyword in keywords:
        df.loc[df["details"].str.contains(keyword, case=False), "category"] = category

# Review the categorization
#print(df[['details', 'category']])


In [11]:
len(df[df['category'].isna()])

68

In [12]:
df[df['category'].isna()]['details'].value_counts().head(40)

details
Nawojowska St. Address                                                                             7
Alfa Partnership                                                                                   2
spp b17 narutowicza  nowy sacz  pol                                                                1
farmedik  nowy sacz  pol                                                                           1
łukasz biskup                                                                                      1
gs lacko sklep nr 7  lacko  pol                                                                    1
smyk spolka akcyjna  nowy sacz  pol                                                                1
euro-net sp. z o.o.  nowy sacz  pol                                                                1
hm  nowy sacz  pol                                                                                 1
ecard s.a., krucza 16, warszawa                                                    

In [13]:
df['category'].fillna('Other', inplace=True)

In [14]:
df['month'] = df['date'].dt.month

In [15]:
# Step 1: Calculate Q1, Q3, and IQR
Q1 = df['amount'].quantile(0.25)
Q3 = df['amount'].quantile(0.75)
IQR = Q3 - Q1

# Step 2: Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Step 3: Filter out outliers
df_filtered = df[(df['amount'] >= lower_bound) & (df['amount'] <= upper_bound)]


In [16]:
df_filtered.pivot_table(index=['month', 'category'], values='amount', aggfunc='sum').sort_values(by=['month', 'amount'], ascending=[True, True]).head(40)

amount
month category                   
1     Supermarket         -986.24
      Personal            -300.00
      Net                 -206.76
      Healthcare          -120.57
      Other                -80.00
      Financial Services   -33.36
      Entertainment        -28.99
      Technology           -23.99
2     Supermarket        -2432.04
      Personal            -860.00
      Other               -347.70
      Home Improvement    -347.58
      Healthcare          -303.83
      Net                 -292.32
      Utility             -206.59
      Technology          -115.97
      Financial Services  -113.27
      Transport            -88.98
      Entertainment        -81.99
      Retail               -81.54
      Food Service         -43.00
      Clothes              -32.69
      Postal Services      -12.80
3     Supermarket        -2024.85
      Net                 -749.24
      Personal            -670.00
      Healthcare          -318.40
      Clothes             -303.04
      Other               -270.84
      Home Improvement    -211.14
      Utility             -189.29
      Food Service        -154.50
      Transport           -143.81
      Retail              -143.76
      dentist             -120.00
      Technology           -85.98
      Entertainment        -81.99
      Financial Services   -30.11
      Postal Services      -12.50
      Alcosol               -8.68

In [17]:
# swap index
dd = df_filtered.pivot_table(index=['category', 'month'], values='amount', aggfunc='sum').loc['Supermarket'].iloc[1:6]
dd

,amount
month,
2,-2432.04
3,-2024.85
4,-2305.59
5,-2563.39
6,-2049.05


In [18]:
dd['amount'].mean()

-2274.9839999999995

In [19]:
dd.loc['mean'] = dd['amount'].mean()

In [20]:
dd

,amount
month,
2,-2432.040
3,-2024.850
4,-2305.590
5,-2563.390
6,-2049.050
mean,-2274.984
